In [1]:
import pandas as pd
train = pd.read_csv("train_data.csv")
test = pd.read_csv("test_data.csv")

In [3]:
train = train.fillna(0)

In [26]:
len(test['WindDir3pm'].unique())

17

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

label_encoder = preprocessing.LabelEncoder()

In [5]:
train_full = pd.get_dummies(train, columns=['Location', 'WindGustDir', 'WindDir9am', 'WindDir3pm'])
test_full = pd.get_dummies(test, columns=['Location', 'WindGustDir', 'WindDir9am', 'WindDir3pm'])

In [37]:
train_full = train_full.drop(['WindDir3pm_0', 'WindDir9am_0', 'WindGustDir_0'], axis=1)

In [38]:
col1 = set(test_full.columns)
col2 = set(train_full.columns)
col2 - col1

{'RainTomorrow'}

In [ ]:
train_full.head()
train_full['RainTomorrow'].value_counts()

In [39]:
from imblearn.over_sampling import SMOTE

os = SMOTE()
train_x = train_full.drop('RainTomorrow', axis=1)
train_y = train_full['RainTomorrow']
os_train_x, os_train_y = os.fit_resample(train_x, train_y)

In [40]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(os_train_x)
train_norm_x = scaler.transform(os_train_x)
train_norm_y = os_train_y

scaler.fit(test_full)
test_norm = scaler.transform(test_full)

In [41]:
x_train, x_test, y_train, y_test = train_test_split(train_norm_x, train_norm_y)

In [ ]:
!pip install catboost

In [10]:
from catboost import CatBoostClassifier

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score
from sklearn.model_selection import KFold, cross_val_score
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier

catboost = CatBoostClassifier()
tree = DecisionTreeClassifier()
classifier = AdaBoostClassifier()
xgb = xgb.XGBClassifier()
randomtree = RandomForestClassifier()

models = [catboost, tree, classifier, xgb, randomtree]

score = []
for model in models:
    model.fit(x_train, y_train)
    y_prediction = model.predict(x_test) 
 
    score.append(f1_score(y_prediction, y_test))

In [ ]:
df = pd.DataFrame({'model': models, 'score': score})
df

In [ ]:
depth = [2, 3, 5, 7]
iterations = [100, 300, 500, 1000]
verbose = [True, False]
params = dict(depth = depth, iterations = iterations, verbose=verbose)
grid = GridSearchCV(catboost, params)
grid.fit(x_train, y_train)

In [55]:
grid.best_params_

{'depth': 7, 'iterations': 1000, 'verbose': True}

In [ ]:
catboost = CatBoostClassifier(depth = 7, iterations = 1000, verbose = True)
catboost.fit(x_train, y_train)
y_pred = catboost.predict(x_test)
accuracy_score(y_pred, y_test)

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score
f1_score(y_pred, y_test)

In [ ]:
os_train_x.head()

In [ ]:
n_estimators = [50, 100, 300, 500]
min_samples_split = [1, 2, 3, 5]
max_depth = [2, 3, 7, 11, 15]
params = dict(n_estimators = n_estimators, min_samples_split = min_samples_split, max_depth = max_depth)
grid = GridSearchCV(randomtree, params)
grid.fit(x_train, y_train)

In [77]:
c = grid.best_params_
c

{'max_depth': 15, 'min_samples_split': 2, 'n_estimators': 300}

In [42]:
randomtree = RandomForestClassifier(n_estimators=300)

In [43]:
randomtree.fit(x_train, y_train)
y_pred = randomtree.predict(x_test)
f1_score(y_pred, y_test)

0.9076823264511696

In [46]:
test_norm = pd.DataFrame(test_norm)

In [47]:
test_norm = test_norm.fillna(0)

In [48]:
test_pred = randomtree.predict(test_norm)

In [58]:
test_pred

array([1., 1., 1., ..., 0., 0., 1.])

In [49]:
f = open('submission.csv', 'w')
for i in test_pred:
    f.write(str(int(i)) + '\n')
f.close()

In [14]:
import xgboost as xgb
clf = xgb.XGBClassifier()
parameters = {
     "eta"    : [0.10, 0.15, 0.20] ,
     "max_depth"        : [ 3, 4, 5, 6],
     "min_child_weight" : [ 1, 3, 5, 7 ],
     "colsample_bytree" : [ 0.3, 0.4, 0.5 , 0.7 ]
     }

grid = GridSearchCV(clf,
                    parameters, n_jobs=4,
                    scoring="neg_log_loss",
                    cv=3)

grid.fit(x_train, y_train)

GridSearchCV(cv=3, estimator=XGBClassifier(), n_jobs=4,
             param_grid={'colsample_bytree': [0.3, 0.4, 0.5, 0.7],
                         'eta': [0.1, 0.15, 0.2], 'max_depth': [3, 4, 5, 6],
                         'min_child_weight': [1, 3, 5, 7]},
             scoring='neg_log_loss')

In [20]:
C = grid.best_params_
C

{'colsample_bytree': 0.7, 'eta': 0.1, 'max_depth': 3, 'min_child_weight': 7}

In [73]:
import xgboost as xgb
clf = xgb.XGBClassifier(max_depth=3, min_child_weight=7, colsample_bytree=0.7, eta = 0.1)
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)
f1_score(y_pred, y_test)

0.8794570968484012